![imagen](data/foto1.png)

## Embeddings

Primero vamos a ver cómo funciona un word embedding de una forma simulada al tiempo que vemos como emplear la capa de embeddings de Keras, luego veremos como hacer sentence embedding utilizando un modulo o modelo preentrenado de Tensorflow.

In [1]:
import numpy as np
import tensorflow as tf

### Word Embeddings

Recuerda que un word embedding transforma las palabras de un texto en un vector de n dimensiones. Veamos como hacerlo con una capa de embeddings, sin entrenar y así podrás ver como instanciarla.

In [2]:
categorias_ejemplo = ["Me","llamo","Iñigo","Montoya","soy","tú","mataste","a","mi","padre"] # Una frase o un vocabulario de ejemplo
pre_conversion = tf.keras.layers.StringLookup() # IMPORTANTE: Hay que convertir nuestro vocabulario a indices
pre_conversion.adapt(categorias_ejemplo) # Y como ya habíamos visto, hay que hacer un fit/adapt
lookup_y_embedding = tf.keras.Sequential([
                                          tf.keras.layers.InputLayer(shape=[], dtype=tf.string),
                                          pre_conversion,
                                          tf.keras.layers.Embedding(input_dim = pre_conversion.vocabulary_size(),
                                                                   output_dim = 2)])
# input_dim -> Tamaño del vocabulario a convertir en vectores de output_dim dimensiones

Este "modelo" no resuelve ningún tipo de problema solo pasa las palabras a traves de la capa de codificación y luego de la embeddings y genera por cadda palabra un vector de 2 dimensiones (output_dim). Pero además como no está entrenada funcionará porque tiene pesos inicializados de forma aleatoria. Es decir que si le pasamos como entrada la variable con la frase de ejemplo...

In [3]:
pre_conversion(categorias_ejemplo)

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 9,  6, 10,  8,  2,  1,  5,  7,  4,  3])>

In [4]:
lookup_y_embedding(np.array(categorias_ejemplo))

<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
array([[-0.04409352,  0.03440963],
       [ 0.01493663,  0.03152325],
       [ 0.03639818, -0.00137614],
       [-0.00677577, -0.00635258],
       [-0.01323632, -0.01922233],
       [-0.0229242 , -0.0473491 ],
       [-0.01399402,  0.02506015],
       [-0.00280821, -0.02898717],
       [ 0.0202817 , -0.00427217],
       [ 0.04536236, -0.00698604]], dtype=float32)>

Nos convierte cada palabra en un embedding (sin sentido)...

Otra forma de hacerlo

In [5]:
frase = "Me llamo Iñigo Montoya"
lookup_y_embedding(np.array(frase.split()))

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[-0.04409352,  0.03440963],
       [ 0.01493663,  0.03152325],
       [ 0.03639818, -0.00137614],
       [-0.00677577, -0.00635258]], dtype=float32)>

Para poder darle valor tendríamos que incluir nuestras dos capas (la codificadora y la de embedding) en un modelo con un objetivo determinado y la capa de embeddings se entrenaría para generar los embeddings que mejor se adapten al problema a solucionar con ese modelo

### Sentences embedding

Vamos a convertir una serie de frases en embeddings. En concreto de 50 dimensiones. Lo haremos utilizando un modelo preentrenado el nnlem-en-dim50 de Google. Internamente es un modelo word embeddings que convierte cada palabra en un embedding de 50 dimensiones y luego calcular el centroide de todos los vectores obtenidos para una frase.

In [6]:
import tensorflow_hub as hub

hub_layer = hub.KerasLayer("https://tfhub.dev/google/nnlm-en-dim50/2") # Así se obtiene el modelo
sentence_embeddings = hub_layer(tf.constant(["To be", "Not to be"]))
sentence_embeddings.numpy().round(2)

array([[-0.25,  0.28,  0.01,  0.1 ,  0.14,  0.16,  0.25,  0.02,  0.07,
         0.13, -0.19,  0.06, -0.04, -0.07,  0.  , -0.08, -0.14, -0.16,
         0.02, -0.24,  0.16, -0.16, -0.03,  0.03, -0.14,  0.03, -0.09,
        -0.04, -0.14, -0.19,  0.07,  0.15,  0.18, -0.23, -0.07, -0.08,
         0.01, -0.01,  0.09,  0.14, -0.03,  0.03,  0.08,  0.1 , -0.01,
        -0.03, -0.07, -0.1 ,  0.05,  0.31],
       [-0.2 ,  0.2 , -0.08,  0.02,  0.19,  0.05,  0.22, -0.09,  0.02,
         0.19, -0.02, -0.14, -0.2 , -0.04,  0.01, -0.07, -0.22, -0.1 ,
         0.16, -0.44,  0.31, -0.1 ,  0.23,  0.15, -0.05,  0.15, -0.13,
        -0.04, -0.08, -0.16, -0.1 ,  0.13,  0.13, -0.18, -0.04,  0.03,
        -0.1 , -0.07,  0.07,  0.03, -0.08,  0.02,  0.05,  0.07, -0.14,
        -0.1 , -0.18, -0.13, -0.04,  0.15]], dtype=float32)

Probemos ahora algunas cosas como por ejemplo obtener la similitud entre sentencias

In [7]:
sentences = ['El Real Madrid lo tiene difícil para ganar al Manchester City.',
 'El Barcelona puede clasificar frente al PSG, si se esfuerza.',
 'Las tropas rusas han tomado Dubroknic.',
 'El ejercito ucraniano se ha replegado']

In [8]:
sentence_embeddings = hub_layer(tf.constant(sentences))

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

for (frase1,vec1),(frase2,vec2) in combinations(zip(sentences,sentence_embeddings.numpy()), r = 2):
    print(frase1,"vs",frase2, cosine_similarity([vec1],[vec2]), np.linalg.norm(vec1-vec2))

El Real Madrid lo tiene difícil para ganar al Manchester City. vs El Barcelona puede clasificar frente al PSG, si se esfuerza. [[0.82780886]] 1.058024
El Real Madrid lo tiene difícil para ganar al Manchester City. vs Las tropas rusas han tomado Dubroknic. [[0.5830585]] 1.505526
El Real Madrid lo tiene difícil para ganar al Manchester City. vs El ejercito ucraniano se ha replegado [[0.6408398]] 1.4179276
El Barcelona puede clasificar frente al PSG, si se esfuerza. vs Las tropas rusas han tomado Dubroknic. [[0.60794]] 1.3915229
El Barcelona puede clasificar frente al PSG, si se esfuerza. vs El ejercito ucraniano se ha replegado [[0.7498946]] 1.1573553
Las tropas rusas han tomado Dubroknic. vs El ejercito ucraniano se ha replegado [[0.5510596]] 1.1434852


Con el coseno tendríamos algún problema con la distancia quedan mejor emparejadas.

In [10]:
question  = "¿Contra quién juega el Barcelona?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []
for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

P:¿Contra quién juega el Barcelona?
R:El Barcelona puede clasificar frente al PSG, si se esfuerza.


In [11]:
question  = "¿Qué hacen los ucranianos?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []
for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

P:¿Qué hacen los ucranianos?
R:El ejercito ucraniano se ha replegado


In [12]:
question  = "¿Qué han pasado en Dubrocnick?"
pregunta = hub_layer(tf.constant([question]))
vec_q = pregunta.numpy()
distancias = []
respuestas = []
for answer,vec_a in zip(sentences,sentence_embeddings.numpy()):
    respuestas.append(answer)
    distancias.append(np.linalg.norm(vec_q-vec_a))
print(f"P:{question}")
print(f"R:{respuestas[np.argmin(distancias)]}")

P:¿Qué han pasado en Dubrocnick?
R:Las tropas rusas han tomado Dubroknic.
